In [2]:
from config_figures import *

In [13]:
plots = pd.read_csv('../../sPlot_data/sPlotOpen_header.txt', sep='\t')
plots

/var/folders/r2/w0bwjv2s0wb_c3q5wpnhbv2h0000gp/T/ipykernel_41140/3841750547.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  plots = pd.read_csv('../../sPlot_data/sPlotOpen_header.txt', sep='\t')


,PlotObservationID,GIVD_ID,Dataset,Continent,Country,Biome,Date_of_recording,Latitude,Longitude,Location_uncertainty,...,Height_shrubs_lowest,Height_herbs_average,Height_herbs_lowest,Height_herbs_highest,SoilClim_PC1,SoilClim_PC2,Resample_1,Resample_2,Resample_3,Resample_1_consensus
0,16,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,...,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
1,17,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,...,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
2,18,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,...,NaN,NaN,NaN,NaN,-3.660,0.546,False,False,True,False
3,20,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,...,NaN,NaN,NaN,NaN,-3.660,0.546,False,True,False,False
4,22,NA-US-014,Aava,North America,United States,Boreal zone,1980-01-01,62.420000,-154.180000,1000.0,...,NaN,NaN,NaN,NaN,-3.660,0.546,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95099,1126749,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2010-08-19,56.287870,27.273862,10.0,...,NaN,NaN,NaN,NaN,-1.599,0.775,True,False,False,True
95100,1126761,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2009-06-19,56.939965,23.598014,10.0,...,NaN,NaN,NaN,NaN,-1.416,0.690,False,False,True,False
95101,1126769,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2009-06-19,56.939965,23.598014,10.0,...,NaN,NaN,NaN,NaN,-1.416,0.690,False,True,False,False
95102,1126774,EU-00-020,European wetlands,Europe,Latvia,Temperate midlatitudes,2003-09-06,57.248222,22.147667,10.0,...,NaN,NaN,NaN,NaN,-1.065,0.849,False,False,True,False


In [6]:
print('sPlot data:')
print(f"{splot_data.size().getInfo()} features in feature collection")
print(f"{splot_data.distinct('Species').size().getInfo()} species")
print(f"{splot_data.distinct('.geo').size().getInfo()} distinct locations")

sPlot data:
1945384 features in feature collection
42676 species
78472 distinct locations


In [4]:
splot_data.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [3.6539300944272632, 44.00009938105824]},
 'id': '0000000000000003ccd1',
 'properties': {'Location_uncertainty': None,
  'PlotObservationID': 482319,
  'Species': 'Aegopodium podagraria'}}

In [7]:
sdm_species = sdms.aggregate_array('system:index').getInfo()
print(len(sdm_species), sdm_species[0:5])

10590 ['Abarema_barbouriana', 'Abarema_brachystachya', 'Abarema_cochleata', 'Abarema_filamentosa', 'Abarema_jupunba']


In [9]:
splot_data = splot_data.filter(ee.Filter.inList('Species', ee.List([s.replace('_', ' ') for s in sdm_species])))
print('sPlot data after filtering for species in SDMs:')
print(f"{splot_data.size().getInfo()} features in feature collection")
print(f"{splot_data.distinct('Species').size().getInfo()} species")
print(f"{splot_data.distinct('.geo').size().getInfo()} distinct locations")

sPlot data after filtering for species in SDMs:
341846 features in feature collection
5939 species
47479 distinct locations


In [55]:
# Merge all observations for each lat-lon (ie. merge observations of same plot at different times)
unique_plot_obs = plot_obs[['Species', 'Latitude', 'Longitude']].drop_duplicates()
unique_plot_obs['presence'] = 1
unique_plot_obs

,Species,Latitude,Longitude,presence
216,Salix alaxensis,62.420000,-154.180000,1
509,Juniperus communis,62.420000,-154.180000,1
833,Alnus alnobetula,62.420000,-154.180000,1
1737,Salix glauca,69.433583,-148.671333,1
2017,Salix glauca,69.674556,-148.721250,1
...,...,...,...,...
1945325,Salix cinerea,56.939965,23.598014,1
1945334,Frangula alnus,56.939965,23.598014,1
1945338,Populus tremula,56.939965,23.598014,1
1945343,Betula pubescens,57.248222,22.147667,1


In [97]:
unique_plot_obs[unique_plot_obs['Species'] == 'Salix alaxensis']

,Species,Latitude,Longitude,presence
216,Salix alaxensis,62.420000,-154.180000,1
3014,Salix alaxensis,69.149000,-148.827000,1
3108,Salix alaxensis,69.132000,-148.835000,1
4307,Salix alaxensis,69.841657,-155.986602,1
4330,Salix alaxensis,69.841510,-155.986613,1
...,...,...,...,...
12941,Salix alaxensis,68.065045,-149.599877,1
12975,Salix alaxensis,68.087804,-149.553895,1
13005,Salix alaxensis,68.135814,-149.443434,1
13022,Salix alaxensis,68.215016,-149.409344,1


In [58]:
pa_matrix = unique_plot_obs.pivot(index=['Latitude','Longitude'], columns='Species', values='presence').fillna(0)
pa_matrix

,Species,Abarema barbouriana,Abarema jupunba,Abarema laeta,Abarema langsdorffii,Abies alba,Abies amabilis,Abies balsamea,Abies cephalonica,Abies concolor,Abies fargesii,...,Ziziphus abyssinica,Ziziphus jujuba,Ziziphus mucronata,Ziziphus nummularia,Ziziphus oenopolia,Ziziphus spina-christi,Zollernia ilicifolia,Zuelania guidonia,Zygia coccinea,Zygia latifolia
Latitude,Longitude,,,,,,,,,,,,,,,,,,,,,
-54.823030,-70.317880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-54.628620,-69.327770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-54.626740,-69.326710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-54.501260,-70.353910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-47.826650,-73.303220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70.594444,24.752778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.600000,27.069444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.611111,29.516667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
species = 'Quercus ithaburensis'
fc = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([f[1]['Longitude'], f[1]['Latitude']]), {'presence': f[1][species]}) for f in pa_matrix.loc[:,species].reset_index().iterrows()])

In [96]:
fc.size().getInfo()

EEException: Request payload size exceeds the limit: 10485760 bytes.

In [93]:
ee.FeatureCollection([ee.Feature(ee.Geometry.Point([f[1]['Longitude'], f[1]['Latitude']]), {'presence': f[1][species]}) for f in pa_matrix.loc[:,species].reset_index().iterrows()])

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-70.31788, -54.82303]},
 'properties': {'presence': 0}}

In [ ]:
'''
def f(el):
    el = ee.List(el)    
    geom = ee.Geometry.Point([ ee.Number(el.get(0)), ee.Number(el.get(1)) ])
    return ee.Feature(geom, {'presence':ee.Number(el.get(2))})
'''

In [ ]:
# -*- coding: utf-8 -*-
import joblib
import numpy
import os
import time
import math

import ee

ee.Initialize()

def f(el):
    el = ee.List(el)    
    geom = ee.Geometry.Point([ ee.Number(el.get(0)), ee.Number(el.get(1)) ])
    return ee.Feature(geom, {'presence':ee.Number(el.get(2))})


folder_in = '../output_files'
folder_out_present = '%s/p_SDM_sPlot_present'%folder_in
if not os.path.exists(folder_out_present):
    os.mkdir(folder_out_present)
folder_out_absent = '%s/p_SDM_sPlot_absent'%folder_in
if not os.path.exists(folder_out_absent):
    os.mkdir(folder_out_absent)

lat_uniquePlotIDs = joblib.load('%s/lat_uniquePlotIDs.jbl'%folder_in)
lon_uniquePlotIDs = joblib.load('%s/lon_uniquePlotIDs.jbl'%folder_in)
loc_uncertainty_uniquePlotIDs = joblib.load('%s/loc_uncertainty_uniquePlotIDs.jbl'%folder_in)
sPlot_presences = joblib.load('%s/sPlot_presences.jbl'%folder_in)
uniqueSpecies = joblib.load('%s/uniqueSpecies.jbl'%folder_in)
uniquePlotIDs = joblib.load('%s/uniquePlotIDs.jbl'%folder_in)

n_plots = len(uniquePlotIDs)

species = 'Quercus ithaburensis'
print(species)

idx = [i for i, s in enumerate(uniqueSpecies) if species == s][0]

presences = sPlot_presences[idx, :]
print('N. present: ', numpy.sum(presences))

if numpy.sum(presences) > 0:

    presence_absence_matrix = numpy.zeros((n_plots, 3))
    
    presence_absence_matrix[:, 0] = lon_uniquePlotIDs
    presence_absence_matrix[:, 1] = lat_uniquePlotIDs
    presence_absence_matrix[:, 2] = presences
    
    myList = []
    for i in range(n_plots):
        myList.append( [presence_absence_matrix[i, 0], 
                        presence_absence_matrix[i, 1],
                        presence_absence_matrix[i, 2]] )
        
    myList_ee = ee.List(myList)
    
    sPlot_points = ee.FeatureCollection(myList_ee.map(f))
    print(str(sPlot_points.size().getInfo()), 'sPlot points: present + absent')
    
    to_export = 1
    if to_export == 1:
        export = ee.batch.Export.table.toAsset(
        		collection = sPlot_points,
        		description = 'test',
        		assetId = 'users/casadeic/sPlot_data/%s'%species.replace(' ','_'))
        export.start()
        print('Export started')
    
        
    sPlot_present = sPlot_points.filterMetadata('presence', 'equals', 1)
    print(str(sPlot_present.size().getInfo()), 'sPlot points: present')

    sPlot_absent = sPlot_points.filterMetadata('presence', 'equals', 0)
    print(str(sPlot_absent.size().getInfo()), 'sPlot points: absent')

    SDM = ee.Image('users/casadeic/treemap/sdms_4models/%s'%species.replace(' ','_'))
    SDM_range = ee.FeatureCollection('users/casadeic/treemap/ranges/%s'%species.replace(' ','_'))
    
    sPlot_present_inrange = sPlot_present.filterBounds(SDM_range);
    print(str(sPlot_present_inrange.size().getInfo()), 'sPlot points: present in SDM range')
    #sPlot_present_inrange = sPlot_present_inrange.distinct('.geo')
    #print(str(sPlot_present_inrange.size().getInfo()), 'sPlot points: present in SDM range, distinct')
    
    
    sPlot_absent_inrange = sPlot_absent.filterBounds(SDM_range)
    print(str(sPlot_absent_inrange.size().getInfo()), 'sPlot points: absent in SDM range')
    #sPlot_absent_inrange = sPlot_absent_inrange.distinct('.geo')
    #print(str(sPlot_absent_inrange.size().getInfo()), 'sPlot points: absent in SDM range, distinct')
    
    SDM_current = SDM.select('covariates_1981_2010')

    p_SDM_sPlot_present_list = []
    p_SDM_sPlot_present = SDM_current.sampleRegions(collection = sPlot_present_inrange, 
                                                    scale = 30, 
                                                    geometries = True)    
    out_feats = p_SDM_sPlot_present.getInfo()['features']
    for out_feat in out_feats:
        properties = out_feat['properties'] 
        p_SDM_sPlot_present_list.append(properties['covariates_1981_2010'])        
    joblib.dump(p_SDM_sPlot_present_list, 
                '%s/%s.jbl'%(folder_out_present, species.replace(' ','_')))
    print('p_SDM_sPlot_present_list length:', len(p_SDM_sPlot_present_list))
    
    p_SDM_sPlot_absent_list = []    
    chunk = 1000    
    collectionSize = int(sPlot_absent_inrange.size().getInfo())
    print(collectionSize, 'points, absent')
    nloops = math.ceil(float(collectionSize)/chunk)
    print(nloops, 'loops')   
    
    for i in range(nloops):
        print(i)
        start = time.time()
        print('Start')
        subset = ee.FeatureCollection(sPlot_absent_inrange.toList(chunk, i*chunk))
        #print(str(subset.size().getInfo()), 'subset points')
        p_SDM_sPlot_absent = SDM_current.sampleRegions(collection = subset, 
                                                       geometries = False) 
        #print(str(p_SDM_sPlot_absent.size().getInfo()), 'p_SDM_sPlot_absent points')
        out_feats = p_SDM_sPlot_absent.getInfo()['features']
        print('It took', time.time()-start)
        
        for out_feat in out_feats:
            properties = out_feat['properties'] 
            p_SDM_sPlot_absent_list.append(properties['covariates_1981_2010'])
        print('p_SDM_sPlot_absent_list length:', len(p_SDM_sPlot_absent_list))
        
    joblib.dump(p_SDM_sPlot_absent_list, 
                '%s/%s.jbl'%(folder_out_absent, species.replace(' ','_')))
    print('p_SDM_sPlot_absent_list length:', len(p_SDM_sPlot_absent_list))